<a href="https://colab.research.google.com/github/LuArtale/SkyPhotoTips/blob/master/HPC/Artale_Luciano_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejercicio 3: Aproximacion del numero "e" utilizando OpenMP

## Introduccion:

Para explorar la plataforma y los recursos que ofrece Colab decidi desarrollar una funcion que calcule una aproximacion del numero e con las funciones de la biblioteca OpenMP 

## Armado del ambiente:

El ambiente de ejecucion utiliza la CPU y sus distintos nucleos para ejecutar este ejercicio. Luego de seleccionar la opcion de ambiente "None" se debe ejecutar la seccion de codigo que contiene el codigo principal en C++. Siguiente a esto se debe ejecutar la seccion que genera el archivo objeto, y finalmente se deben indicar los parametros deseados en la ultima seccion y ejecutar el ejercicio propiamente dicho.

## Codigo principal para OpenMP en C++

In [1]:
# Codigo Python, que tiene el código C de la ejecución Axpy.
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP  
#include <math.h>  

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_NUME_SEC      2
#define HTH_NUME_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c,j;
  float val_e_sec, val_e_omp;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 3 )
  {
      std::cerr<< " Error en los parametros de indicar: (Nivel de aproximacion), (Ciclos de ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  int nivel_de_aproximacion = atoi( argv[1] );
  int ciclos = atoi( argv[2] );

  // --------------------------------------------
  // Realizo la función e en forma secuencial.

  TIEMPO_INI( HTH_NUME_SEC )

  for(c=0;c<ciclos;c++)
  {
    val_e_sec = 1.0;
    
    for(i=1;i<=nivel_de_aproximacion;i++)
    {
      float divRes = (float)1 / (float)i;
      val_e_sec *= (float)((float)1 + divRes);
      //std::cout<<"Ciclo: "<<c<<", Valor "<<i<<" de numero e = "<<val_e_sec<<", divRes = "<<divRes<<std::endl;
    }

    //std::cout<<"Ciclo: "<<c<<", Valor numero e = "<<val_e_sec<<std::endl;
  }

  TIEMPO_FIN( HTH_NUME_SEC )

  // --------------------------------------------
  // Realizo la función e con OpenMP.

  TIEMPO_INI( HTH_NUME_OMP )

  for(c=0;c<ciclos;c++)
  {
    val_e_omp = 1.0;

    //SECCION PARALELA//
    #pragma omp parallel for reduction(*: val_e_omp)
    for(i=1;i<=nivel_de_aproximacion;i++)
    {
      float divRes = (float)1 / (float)i;
      val_e_sec *= (float)((float)1 + divRes);
      //std::cout<<"Ciclo: "<<c<<", Valor "<<i<<" de numero e = "<<val_e_omp<<", divRes = "<<divRes<<std::endl;
    }
    //SECCION PARALELA//

    //std::cout<<"Ciclo: "<<c<<", Valor numero e = "<<val_e_omp<<std::endl;
  }

  TIEMPO_FIN( HTH_NUME_OMP )

  // --------------------------------------------
  // Muestro los resultados.
  std::cout<<"Resultados:"<<std::endl;
  std::cout<<"Nivel de aproximacion: "<<nivel_de_aproximacion<<std::endl;
  std::cout<<"Aproximaciones del numero e: "<<std::endl;
  std::cout<<"* Calculado secuencialmnte: "<<val_e_sec<<std::endl;
  std::cout<<"* Calculado paralelamente con OpenMP: "<<val_e_omp<<std::endl;

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<std::endl;
 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_NUME_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_NUME_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_NUME_SEC)<<" / "<<TIEMPO_GET(HTH_NUME_OMP)<<" = "<<TIEMPO_GET(HTH_NUME_SEC)/TIEMPO_GET(HTH_NUME_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_NUME_SEC)/TIEMPO_GET(HTH_NUME_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_NUME_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_NUME_SEC)<<" = "<<TIEMPO_GET(HTH_NUME_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_NUME_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP)<<" - "<<TIEMPO_GET(HTH_NUME_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP))-TIEMPO_GET(HTH_NUME_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_NUME_OMP) = TIEMPO_GET(HTH_NUME_SEC) / 2;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_NUME_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_NUME_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_NUME_SEC)<<" / "<<TIEMPO_GET(HTH_NUME_OMP)<<" = "<<TIEMPO_GET(HTH_NUME_SEC)/TIEMPO_GET(HTH_NUME_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_NUME_SEC)/TIEMPO_GET(HTH_NUME_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_NUME_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_NUME_SEC)<<" = "<<TIEMPO_GET(HTH_NUME_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_NUME_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP)<<" - "<<TIEMPO_GET(HTH_NUME_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_NUME_OMP))-TIEMPO_GET(HTH_NUME_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

"""
text_file = open("code_axpy.cpp", "w")
text_file.write(code)
text_file.close()

## Generar archivo objeto mediante el compilador de c++:

In [2]:
!g++ -o axpy -fopenmp code_axpy.cpp

## Ejecutar ejercicio compilado:

Indicar como parametro de la constante OMP_NUM_THREADS la cantidad de hilos y como parametros de llamado a la funcion el nivel de aproximacion (numero del 1 a 1000000) y cantidad de ciclos de ejecucion

In [3]:
#@title Parámetros de ejecución { vertical-output: true }
# Parametros
cantidad_de_hilos = 4#@param {type: "number"}
nivel_de_aproximacion = 3#@param {type: "number"}
ciclos_de_ejecucion = 10#@param {type: "number"}

%env OMP_NUM_THREADS=cantidad_de_hilos
!./axpy nivel_de_aproximacion ciclos_de_ejecucion

env: OMP_NUM_THREADS=cantidad_de_hilos

libgomp: Invalid value for environment variable OMP_NUM_THREADS
Resultados:
Nivel de aproximacion: 0
Aproximaciones del numero e: 
* Calculado secuencialmnte: -8.81734e+37
* Calculado paralelamente con OpenMP: 3.0907e-41

Valores Reales  :
Tiempo TOTAL     : 0.0600815 [ms]
Tiempo axpy Sec  : 0 [ms]
Tiempo axpy Omp  : 0 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0 / 0 = -nan
Eficiencia       : SpeedUp/nro procesadores            : -nan / 2 = -nan
Coste Sec        : nro procesadores*Tiempo             : 1 * 0 = 0
Coste Omp        : nro procesadores*Tiempo             : 2 * 0 = 0
Funcion Overhead : Coste Omp - tiempo Secuencial       : 0 - 0 = 0

Valores Ideal: 
Tiempo axpy Sec  : 0 [ms]
Tiempo axpy Omp  : 0 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0 / 0 = -nan
Eficiencia       : SpeedUp/nro procesadores            : -nan / 2 = -nan
Coste Sec        : nro procesadores*Tiempo             : 1 * 0 = 0
Coste Omp 

## Bibliografia:



*   https://soymatematicas.com/numero-e/#:~:text=Al%20igual%20que%20%CF%80%2C%20el,de%20e%20es%202'7182818284%20%E2%80%A6
*   https://github.com/wvaliente/SOA_HPC/blob/main/Ejercicios/Prueba%201%20-%20Vectores%20-%20OpenMP.ipynb

